##Setup

In [74]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

##Import dataset

In [ ]:
Dataframe_listings = pd.read_csv('/Listings.csv', encoding='latin1', parse_dates=['host_since'])

#pd.set_option('display.max_columns', None)

Dataframe_listings.info()

## Filter Sur les annonces Parisiennes puis conserver que certaines colonnes

In [ ]:
Dataframe_listings_filter = Dataframe_listings[Dataframe_listings['city'] == 'Paris']
Dataframe_listings_filter = Dataframe_listings_filter[['host_since', 'neighbourhood', 'city', 'accommodates', 'price']]

Dataframe_listings_filter

## Vérifier la qualité des données

In [ ]:
Dataframe_listings_filter.isnull().sum()


## Calculer le minimum, le maximum et la moyenne pour les champs numériques

In [ ]:
Dataframe_listings_filter[['accommodates', 'price']].describe()



## Regrouper les annonces parisiennes par « quartier » et calcule le prix moyen

In [ ]:
paris_listing_quartier = Dataframe_listings_filter[['neighbourhood', 'price']]

paris_listing_quartier = paris_listing_quartier.groupby('neighbourhood').mean().sort_values('price', ascending= False)

paris_listing_quartier = paris_listing_quartier.reset_index()

paris_listing_quartier['price']= paris_listing_quartier['price'].round(decimals=1)

paris_listing_quartier

## Regroupez par colonne « hébergement » et ajoutez le prix moyen pour chaque valeur de « hébergement »

In [ ]:
paris_listings_accomodations = Dataframe_listings_filter[['accommodates', 'price']]

paris_listings_accomodations = paris_listings_accomodations.groupby('accommodates').mean().sort_values('price', ascending= False)

paris_listings_accomodations = paris_listings_accomodations.reset_index()

paris_listings_accomodations['price']= paris_listings_accomodations['price'].round(decimals=1)

paris_listings_accomodations


## Regrouper par année « host_since » et calculez le prix moyen et le nombre de lignes représentant le nombre de nouveaux hôtes.

In [ ]:
Dataframe_listings_filter['year'] = Dataframe_listings_filter['host_since'].dt.year
paris_listings_over_time = Dataframe_listings_filter[['year', 'price']]
paris_listings_over_time = paris_listings_over_time.groupby('year').mean()
paris_listings_over_time

##Créez un graphique à barres horizontales du prix moyen par quartier à Paris

In [31]:
fig = px.bar(paris_listing_quartier,
             y='neighbourhood',
             x='price',
             text='price',
             color='neighbourhood',
             title="Prix moyen par quartier")
fig.show()

## Créer un grapique à barres horizontales du prix moyen par acceuil

In [29]:
fig = px.bar(
    paris_listings_accomodations,
    y ='price',
    x='accommodates',
    color= 'accommodates',
    text='price',
    title= 'Prix moyen par acceuil')

fig.show()

## Créez deux graphiques linéaires : un montrant le nombre de nouveaux hôtes au fil du temps et un montrant le prix moyen.

In [ ]:
# Calcul du nombre de hôtes par an
dataframe_filter_bis = Dataframe_listings[['host_id', "host_since"]]

dataframe_filter_bis['year']= dataframe_filter_bis['host_since'].dt.year

dataframe_filter_bis_count = dataframe_filter_bis.groupby('year').count().reset_index()

dataframe_filter_bis_count = dataframe_filter_bis_count[['year', 'host_id']]

# Calcul du prix moyen par an
dataframe_filter_ter = Dataframe_listings[['host_since', 'price']]

dataframe_filter_ter['year']= dataframe_filter_ter['host_since'].dt.year

dataframe_filter_ter_mean = dataframe_filter_ter.groupby('year').mean().reset_index()

dataframe_filter_ter_mean = dataframe_filter_ter_mean[['year', 'price']]

# Fusion des deux dataframes

dataframe_two_linear_graph = dataframe_filter_bis_count.merge(dataframe_filter_ter_mean, how ='left', on='year')

dataframe_two_linear_graph['price']= dataframe_two_linear_graph['price'].round(decimals =0)

dataframe_two_linear_graph

In [84]:
#Visualisation
fig = make_subplots(rows=1, cols=2, subplot_titles=("Nombre de clients par année", "Prix moyen par année"))

fig.add_trace(
    go.Scatter(x=dataframe_two_linear_graph['year'],
               y=dataframe_two_linear_graph['host_id']),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=dataframe_two_linear_graph['year'],
               y=dataframe_two_linear_graph['price']),

    row=1, col=2
)

fig.show()